# KarabukWildfire2025 – Analiz Defteri

Tek defter akışı: GEE init, AOI, Sentinel‑2 ön işleme, NDVI/NBR, dNDVI/dNBR, şiddet ve folium haritalar.
Önkoşullar: `pip install -r requirements.txt` ve bir kez `earthengine authenticate`.


In [ ]:
# Importlar
import os
from IPython.display import IFrame, display
import pandas as pd

from src.pipeline import run_pipeline
from src.utils import ee_init


In [ ]:
# Earth Engine proje kimliği (opsiyonel) ve erken başlatma
ee_project = 'karabukwildfire2025'
print('EE_PROJECT =', ee_project or '(not set; you may be prompted to authenticate)')
try:
    ee_init(project=ee_project)
    print('Earth Engine initialized.')
except Exception as e:
    print('Earth Engine init error:', e)


In [ ]:
# Parametreler
pre_start, pre_end = '2025-07-10', '2025-07-25'
post_start, post_end = '2025-07-26', '2025-08-10'
aoi_path = 'data/aoi.geojson'  # yoksa varsayılan Karabük bbox kullanılır
out_dir = 'results'

print('AOI exists? ', os.path.exists(aoi_path))
print('OUT dir   : ', out_dir)
pre_start, pre_end, post_start, post_end


In [ ]:
# Çalıştır
outputs = run_pipeline(
    pre_start=pre_start, pre_end=pre_end,
    post_start=post_start, post_end=post_end,
    aoi_geojson=aoi_path, out_dir=out_dir,
    project=ee_project
)
outputs


In [ ]:
# Haritaları defter içinde görüntüle (opsiyonel)
display(IFrame(src=outputs['pre_ndvi_map'], width='100%', height=480))
display(IFrame(src=outputs['post_ndvi_map'], width='100%', height=480))
display(IFrame(src=outputs['pre_nbr_map'], width='100%', height=480))
display(IFrame(src=outputs['post_nbr_map'], width='100%', height=480))
display(IFrame(src=outputs['dnbr_map'], width='100%', height=600))
display(IFrame(src=outputs['severity_map'], width='100%', height=600))


In [ ]:
# Özet istatistikler
pd.read_csv(outputs['summary_csv'])


In [ ]:
# Rapor PNG'leri (analiz katmanları + doğal renk)
from src.gee.aoi import get_aoi
from src.gee.preprocess import prepare_composite
from src.gee.indices import with_indices
from src.gee.change import compute_diffs, classify_dnbr
from src.visualize import export_report_pngs, export_truecolor_pngs
from IPython.display import HTML, display
import os, base64

def show_png(path):
    web = 'files/' + path.replace('\\','/').replace('\\\\','/')
    return HTML(f"<img src='{web}' style='max-width:100%;height:auto;'/>")

def show_png_inline(path):
    with open(path,'rb') as f:
        data = base64.b64encode(f.read()).decode('ascii')
    return HTML(f"<img src='data:image/png;base64,{data}' style='max-width:100%;height:auto;'/>")

aoi = get_aoi(aoi_path)
pre = with_indices(prepare_composite(aoi, pre_start, pre_end))
post = with_indices(prepare_composite(aoi, post_start, post_end))
diffs = compute_diffs(pre, post)
severity = classify_dnbr(diffs['dNBR'])

pngs = export_report_pngs(pre=pre, post=post, diffs=diffs, severity=severity, aoi=aoi, out_dir=out_dir)
rgbs = export_truecolor_pngs(pre=pre, post=post, aoi=aoi, out_dir=out_dir)

order = [
  ('Pre RGB', rgbs.get('pre_rgb_png')),
  ('Post RGB', rgbs.get('post_rgb_png')),
  ('Pre NDVI', pngs.get('pre_ndvi_png')),
  ('Post NDVI', pngs.get('post_ndvi_png')),
  ('dNBR', pngs.get('dnbr_png')),
  ('Severity', pngs.get('severity_png')),
]
for title, p in order:
    if not p:
        continue
    try:
        display(HTML(f"<h4 style='margin:8px 0'>{title}</h4>"))
        display(show_png(p))
    except Exception:
        display(show_png_inline(p))
